In [1]:
import cv2 as cv
import numpy as np
import os

In [2]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.3,fy=0.3)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [51]:
def extrage_careu(image):
    image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    image = image[500:3500, 600:2600]
    image_m_blur = cv.medianBlur(image,3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
#     show_image('image_sharpened',image_sharpened)
    _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)

    kernel = np.ones((3, 3), np.uint8)
    thresh = cv.erode(thresh, kernel)
#     show_image('image_thresholded',thresh)

   #600, 500   2600,3500 

    
    edges =  cv.Canny(thresh ,250,450)
#     edges =  cv.Canny(thresh ,100,1000)
#     show_image('edges',edges)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 700
    height = 1000
    
    image_copy = cv.cvtColor(image.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
#     show_image("detected corners",image_copy)

    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    result = cv.warpPerspective(image, M, (width, height))
    result = cv.cvtColor(result,cv.COLOR_GRAY2BGR)
    
    return result

In [12]:
lines_horizontal=[]
for i in range(0,1001,100):
    l=[]
    l.append((0,i))
    l.append((699,i))
    lines_horizontal.append(l)

In [18]:
lines_vertical=[]
for i in range(0,701,100):
    l=[]
    l.append((i,0))
    l.append((i,999))
    lines_vertical.append(l)

In [52]:
files=os.listdir('data/train')
for file in files:
    if file[-3:]=='jpg':
        img = cv.imread('data/train/'+file)
        result=extrage_careu(img)
        for line in  lines_vertical : 
            cv.line(result, line[0], line[1], (0, 255, 0), 5)
        for line in  lines_horizontal : 
            cv.line(result, line[0], line[1], (0, 0, 255), 5)
        show_image('img',result)

In [31]:
def determina_configuratie_careu_ox(thresh,lines_horizontal,lines_vertical):
    matrix = np.empty((10,12), dtype='str')
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0] + 20
            y_max = lines_vertical[j + 1][1][0] - 20
            x_min = lines_horizontal[i][0][1] + 20
            x_max = lines_horizontal[i + 1][1][1] - 20
            
            patch = thresh[x_min:x_max, y_min:y_max].copy()
            Medie_patch=np.mean(patch)
            if Medie_patch>0:
                matrix[i][j]='x'
            else:
                matrix[i][j]='o'
    return matrix

In [16]:
def vizualizare_configuratie(result,matrix,lines_horizontal,lines_vertical):
    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            if matrix[i][j] == 'x': 
                cv.rectangle(result, (y_min, x_min), (y_max, x_max), color=(255, 0, 0), thickness=5)

In [24]:
files=os.listdir('data/train')
for file in files:
    if file[-3:]=='jpg':
        img = cv.imread('data/train/'+file)
        result=extrage_careu(img)
        _, thresh = cv.threshold(result, 100, 255, cv.THRESH_BINARY_INV)
        matrice=determina_configuratie_careu_ox(thresh,lines_horizontal,lines_vertical)
        vizualizare_configuratie(result,matrice,lines_horizontal,lines_vertical)
        show_image('img',result)

In [27]:
files=os.listdir('data/train')
X_test = []
hit_list = []
for file in files:
    if file[-3:]=='txt' and file != 'configuratii.txt':
        with open('data/train/'+ file) as f:
            continutFisier = f.read()
            split = continutFisier.split("\n")
            hit_list.append(split[1:(int(split[0])+1)])
print(hit_list)
print(hit_list[0])
print(hit_list[0][0])
print(hit_list[0][0][0])
print(type(hit_list[0][0][0]))
print(type(int(hit_list[0][0][0])))
print(int(hit_list[0][0][0])-1)

[['1A', '2B', '5E', '8C', '9F'], ['1H', '3C', '7B', '8E', '9A', '11A', '12B'], ['1C', '3D', '4F', '4G', '5A', '6D'], ['1A', '1C', '1D', '7D', '8E', '8F', '9E', '10E'], ['2D', '2E', '7H', '9D']]
['1A', '2B', '5E', '8C', '9F']
1A
1
<class 'str'>
<class 'int'>
0


In [28]:
def letter_to_number(l):
    if l == "A":
        return 1
    elif l == "B":
        return 2
    elif l == "C":
        return 3
    elif l == "D":
        return 4
    elif l == "E":
        return 5
    elif l == "F":
        return 6
    elif l == "G":
        return 7
    elif l == "H":
        return 8
    elif l == "I":
        return 9
    elif l == "J":
        return 10

In [55]:
files=os.listdir('data/train')
i=0
for file in files:
    if file[-3:]=='jpg':
        img = cv.imread('data/train/'+file)
        result=extrage_careu(img)
        _, thresh = cv.threshold(result, 100, 255, cv.THRESH_BINARY_INV)
        matrice=determina_configuratie_careu_ox(thresh,lines_horizontal,lines_vertical)
       
    if file[-3:]=='txt' and file != 'configuratii.txt':
        with open('data/train/'+ file) as f:
            continutFisier = f.read()
            split = continutFisier.split("\n")
            hits = split[1:(int(split[0])+1)]
            
            
        with open('out'+ str(i+1) +'.txt','w') as g:
            for position in hits:
                if (position[0] == '1' and position[1] == '0') or (position[0] == '1' and position[1] == '1') or (position[0] == '1' and position[1] == '2'):
                    if matrice[int(position[0:1])-1][letter_to_number(position[2])-1] == 'x':
                        label = 1
                    else:
                        label = 0
                
                elif matrice[int(position[0])-1][letter_to_number(position[1])-1] == 'x':
                    label = 1
                else:
                    label = 0

                print(position, label, file = g)
        i+=1
print("fisierele de output au fost create in directory-ul cu .ipynb")

fisierele de output au fost create in directory-ul cu .ipynb
